In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Sequence, List
from sklearn.model_selection import train_test_split, StratifiedKFold
SEED = 42
GPU_ID = 0

In [2]:
train_df = pd.read_csv("/kaggle/input/fall-ml2-mipt-2022/train.csv", index_col="Id")

In [3]:
train_df["Category"] = train_df["Category"].apply(int)

In [4]:
train_df.sample(5)

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_391,x_392,x_393,x_394,x_395,x_396,x_397,x_398,x_399,Category
Id,,,,,,,,,,,,,,,,,,,,,
1811,-0.000233,0.000792,0.012093,0.002450,-0.009154,0.005280,0.012169,0.005127,1.176650,0.003068,...,-1.246578,0.127770,-0.624698,0.010376,-0.816160,0.022156,0.062792,0.152688,-0.001649,9
8530,-0.000695,-0.005563,-0.004448,-0.010461,-0.006589,-0.008881,-0.008745,0.016755,0.002001,-0.016571,...,-1.080785,-0.020550,-0.931434,-0.029828,-0.846592,-0.000154,-0.859487,-1.731615,-0.010569,13
3515,0.001228,0.008510,-0.004602,0.015169,-0.001202,1.356137,-0.001908,0.002606,0.009053,0.013155,...,-0.745583,0.022914,-1.232140,-0.001934,1.048430,0.039204,0.035280,-1.558216,0.001258,4
67,-0.003401,-0.025186,0.001077,0.012092,0.013850,0.004742,0.006721,-0.013682,0.013626,0.030928,...,-0.254494,-0.006869,-0.783260,-0.002593,0.921972,-0.024300,-1.074321,-0.465769,-0.000664,10
641,0.011070,-0.008515,-0.012518,0.020023,0.533146,0.006375,-0.010719,0.004545,0.011045,0.002347,...,1.181900,0.144594,0.615429,-0.009018,-1.339429,0.097724,-0.069085,-0.167476,0.045938,28


In [5]:
train_df.describe()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_391,x_392,x_393,x_394,x_395,x_396,x_397,x_398,x_399,Category
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.019503,0.006501,0.015427,0.031731,0.033968,0.024515,0.006047,0.023426,0.024212,0.030499,...,-0.001936,0.000199,-0.007278,0.000172,-0.012318,0.000856,-0.007320,-0.000230,0.000224,17.062500
std,0.361609,0.347227,0.344725,0.368493,0.389563,0.368459,0.331870,0.373213,0.345279,0.384832,...,1.004800,0.045655,0.998830,0.032463,1.006192,0.037690,0.993976,1.002536,0.047652,9.724107
min,-3.164112,-3.010361,-3.308394,-2.881321,-3.439496,-2.698778,-3.180149,-3.014124,-2.824733,-3.439037,...,-3.634521,-0.325977,-3.938113,-0.270285,-3.523945,-0.265358,-3.933017,-3.773770,-0.297632,0.000000
25%,-0.007520,-0.007291,-0.007233,-0.007286,-0.006951,-0.007153,-0.007183,-0.007398,-0.007143,-0.006988,...,-0.689239,-0.021368,-0.688154,-0.012640,-0.700943,-0.014773,-0.677760,-0.672713,-0.022221,9.000000
50%,0.000691,0.000188,0.000622,0.000758,0.000727,0.000653,0.000171,0.000581,0.000663,0.000931,...,-0.011012,0.000073,0.001013,0.000126,-0.013112,0.000330,-0.023013,-0.000571,-0.000253,17.000000
75%,0.008769,0.008157,0.008546,0.009497,0.009073,0.008897,0.008016,0.008989,0.009019,0.009359,...,0.688578,0.021935,0.669180,0.012842,0.669700,0.016359,0.663634,0.668707,0.022257,27.000000
max,4.044388,3.526068,3.014865,2.999935,3.055828,3.344133,3.282431,2.998927,3.491133,3.406931,...,3.909980,0.246096,4.278834,0.238513,3.801937,0.247514,3.949601,3.671366,0.371345,31.000000


In [6]:
def stratified_k_fold(ids: Sequence, target: Sequence, n_splits: int=3, random_state=42) -> List[List[List]]:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    split = []
    for train, val_test in skf.split(ids, target):
        val_test_skf = StratifiedKFold(2)
        _target = [target[i] for i in val_test]
        split.append([train, *next(val_test_skf.split(val_test, _target))])

    return split

In [7]:
split = stratified_k_fold(train_df.index.to_list(), train_df["Category"], 5)

In [8]:
len(split[0][0]), len(split[0][1]), len(split[0][2])

(8000, 1000, 1000)

In [9]:
import xgboost

In [10]:
models, val_preds = [], []
eval_metrics = ['mlogloss', 'auc', 'merror']
results = []

n_estimators = 1000
max_depth = 5
reg_alpha = 10

for i, (train_ids, val_ids, test_ids) in enumerate(split):
    print(f"FOLD #{i}")
    model = xgboost.XGBClassifier(n_estimators=n_estimators, tree_method='gpu_hist', reg_alpha=reg_alpha, learning_rate=0.01, gpu_id=GPU_ID,
                                  objective="multi:softprob", num_class=32, eval_metric=eval_metrics, max_depth=max_depth, seed=SEED)
    model.fit(train_df.loc[train_ids][[f"x_{i}" for i in range(400)]], train_df.loc[train_ids]["Category"], verbose=False,
             eval_set=[(train_df.loc[val_ids][[f"x_{i}" for i in range(400)]], train_df.loc[val_ids]["Category"])])
    models.append(model)
    results.append(model.evals_result())
    model.save_model(f'model_{i}.json')
    
    print({k: m[-1] for k, m in results[i]["validation_0"].items()})

FOLD #0
{'mlogloss': 1.5387025700956583, 'auc': 0.96740864776974, 'merror': 0.269}
FOLD #1
{'mlogloss': 1.5120405847132206, 'auc': 0.9665853104260035, 'merror': 0.259}
FOLD #2
{'mlogloss': 1.5570230400264262, 'auc': 0.9628748102794532, 'merror': 0.279}
FOLD #3
{'mlogloss': 1.5411496153622866, 'auc': 0.9652705562607595, 'merror': 0.268}
FOLD #4
{'mlogloss': 1.5402412012517452, 'auc': 0.9661541042258965, 'merror': 0.252}


In [11]:
test_df = pd.read_csv("/kaggle/input/fall-ml2-mipt-2022/test.csv", index_col="Id")
probas = np.mean([model.predict_proba(test_df) for model in models], axis=0)
test_df["Category"] = np.argmax(probas, axis=1)

In [12]:
test_df[["Category"]].to_csv("submission.csv")